# Python 知识

## 元类

### type

Python 有个内置类型 `type`，其是所有类的类型（注意实例的类型是类）。

`type` 有两个作用：
- 查看对象类型：`type(obj)`
- 动态创建类 `type(name, bases, dict)`
  - `name（字符串）`：新类的名称。
  - `bases（元组）`：新类需要继承的父类组成的元组。
  - `dict（字典）`：包含属性和方法。键是属性或方法的名称（字符串），字值是对应的属性值或函数对象。

In [1]:
def fn(self, name='world'):
    print('Hello, %s.' % name)

Hello = type('Hello', (object,), dict(hello=fn))
h = Hello()
h.hello()

Hello, world.


### `*()` 等价于 `type(*).__call__`

注意 `type` 有方法 `__call__`：先调用 `__new__` 再调用 `__init__`。

### 类创建过程

当 Python 遇到类定义时，例如
```python
class MyClass(BaseClass, metaclass=MyMeta):
    attr = "value"
    
    def method(self):
        pass
```
会经历以下步骤：
- 收集信息
  - 类名：`MyClass`
  - 基类：`(BaseClass,)`
  - 命名空间：`{'attr': 'value', 'method': <function>}`
  - 元类：`MyMeta`
    - 元类查找顺序
      - 显式指定的 `metaclass` 参数
      - 基类的元类 
      - 默认的 `type`
- 调用元类：`   MyClass = MyMeta('MyClass', (BaseClass,), {'attr': 'value', 'method': <function>})`

In [8]:
class DetailedMeta(type):
    def __call__(cls, *args, **kwargs):
        print(f"\n=== DetailedMeta.__call__ ===")
        print(f"正在创建类的实例")
        print(f"类: {cls}")
        print(f"参数: {args}, {kwargs}")
        
        # 当创建类的实例时调用这里
        instance = super().__call__(*args, **kwargs)
        print(f"实例创建完成: {instance}")
        return instance
    
    def __new__(mcs, name, bases, namespace):
        print(f"\n=== DetailedMeta.__new__ ===")
        print(f"正在创建类: {name}")
        print(f"元类: {mcs}")
        print(f"基类: {bases}")
        print(f"原始命名空间: {list(namespace.keys())}")
        
        # 修改命名空间
        namespace['_created_by_meta'] = True
        namespace['_meta_name'] = mcs.__name__
        
        print(f"修改后命名空间: {list(namespace.keys())}")
        
        # 创建类对象
        cls = super().__new__(mcs, name, bases, namespace)
        print(f"类对象创建完成: {cls}")
        
        return cls
    
    def __init__(cls, name, bases, namespace):
        print(f"\n=== DetailedMeta.__init__ ===")
        print(f"正在初始化类: {cls}")
        print(f"类名: {name}")
        
        # 添加类级别的属性
        cls._creation_info = {
            'name': name,
            'bases': [base.__name__ for base in bases],
            'methods': [key for key in namespace.keys() 
                       if callable(namespace[key]) and not key.startswith('__')]
        }
        
        super().__init__(name, bases, namespace)

# 基类
class BaseClass:
    def base_method(self):
        return "base method"

# 现在让我们观察创建过程
print("开始创建类...")
MyClass = DetailedMeta('MyClass', (BaseClass,), {
    'attr': 'value',
    'my_method': lambda self: "my method result"
})

print(f"\n创建的类: {MyClass}")
print(f"类的属性: {dir(MyClass)}")
print(f"创建信息: {MyClass._creation_info}")

# 创建实例
print(f"\n开始创建实例...")
obj = MyClass()
print(f"实例: {obj}")

开始创建类...

=== DetailedMeta.__new__ ===
正在创建类: MyClass
元类: <class '__main__.DetailedMeta'>
基类: (<class '__main__.BaseClass'>,)
原始命名空间: ['attr', 'my_method']
修改后命名空间: ['attr', 'my_method', '_created_by_meta', '_meta_name']
类对象创建完成: <class '__main__.MyClass'>

=== DetailedMeta.__init__ ===
正在初始化类: <class '__main__.MyClass'>
类名: MyClass

创建的类: <class '__main__.MyClass'>
类的属性: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_created_by_meta', '_creation_info', '_meta_name', 'attr', 'base_method', 'my_method']
创建信息: {'name': 'MyClass', 'bases': ['BaseClass'], 'methods': ['my_method']}

开始创建实例...

=== DetailedMeta.__call__ ===
正在创建类的实例
类: <c

### 例子：自定义单例模式元类

In [2]:
class SingletonMeta(type):
    """单例模式元类"""
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Database(metaclass=SingletonMeta):
    def __init__(self):
        self.connection = "Connected to DB"

# 测试单例模式
db1 = Database()
db2 = Database()
print(db1 is db2)  # True - 同一个实例

True


### 例子：使用函数作为元类

In [3]:
def debug_meta(name, bases, attrs):
    """调试元类：打印类创建信息"""
    print(f"创建类: {name}")
    print(f"基类: {bases}")
    print(f"属性: {list(attrs.keys())}")
    
    # 为所有方法添加调试装饰器
    for key, value in attrs.items():
        if callable(value) and not key.startswith('__'):
            attrs[key] = debug_wrapper(value)
    
    return type(name, bases, attrs)

def debug_wrapper(func):
    def wrapper(*args, **kwargs):
        print(f"调用方法: {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

class Calculator(metaclass=debug_meta):
    def add(self, a, b):
        return a + b
    
    def multiply(self, a, b):
        return a * b

# 输出：
# 创建类: Calculator
# 基类: ()
# 属性: ['__module__', '__qualname__', 'add', 'multiply']

calc = Calculator()
result = calc.add(2, 3)  # 调用方法: add

创建类: Calculator
基类: ()
属性: ['__module__', '__qualname__', '__firstlineno__', 'add', 'multiply', '__static_attributes__']
调用方法: add


### 例子：自动属性注册

In [4]:
class AutoRegisterMeta(type):
    """自动注册类属性的元类"""
    registry = {}
    
    def __new__(mcs, name, bases, attrs):
        cls = super().__new__(mcs, name, bases, attrs)
        
        # 自动注册所有非私有属性
        for attr_name, attr_value in attrs.items():
            if not attr_name.startswith('_'):
                mcs.registry[f"{name}.{attr_name}"] = attr_value
        
        return cls

class Config(metaclass=AutoRegisterMeta):
    database_url = "postgresql://localhost/mydb"
    cache_timeout = 300
    debug_mode = True

# 自动注册的配置
print(AutoRegisterMeta.registry)

{'Config.database_url': 'postgresql://localhost/mydb', 'Config.cache_timeout': 300, 'Config.debug_mode': True}


# 代码分析